In [10]:
import pandas as pd
import os
from llama_index.core import Document
from llama_index.core import Settings # Importar Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter # Para dividir el texto
from llama_index.core import VectorStoreIndex
import logging
import sys

# Configuración básica de logging para ver el progreso
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# --- Configuración Global de LlamaIndex ---
print("Configurando el modelo de embeddings...")
try:
    # Usaremos un modelo de embeddings popular y eficiente
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    print("Modelo de embeddings configurado.")
except Exception as e:
    print(f"Error configurando el modelo de embeddings: {e}")
    print("Asegúrate de tener conexión a internet para descargar el modelo si es la primera vez.")
    exit()

# --- 1. Cargar y Combinar los CSVs (SOLO TRAIN Y VALIDATION) ---
output_dir = "../../data/unfair_tos_data"  # El directorio donde guardaste los CSVs
csv_files_to_combine = { # Definimos explícitamente los archivos a usar
    "train": os.path.join(output_dir, "unfair_tos_train.csv"),
    "validation": os.path.join(output_dir, "unfair_tos_validation.csv")
}

all_dfs = []
print("\nCargando y combinando archivos CSV (train y validation)...")
for split_name, file_path in csv_files_to_combine.items(): # Usamos el nuevo diccionario
    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            # Ya no necesitamos 'original_split' en los metadatos finales,
            # pero podríamos mantenerla aquí si quisiéramos diferenciar en el df_full
            # df['original_split'] = split_name
            all_dfs.append(df)
            print(f"Archivo '{file_path}' cargado (filas: {len(df)}).")
        except Exception as e:
            print(f"Error cargando el archivo {file_path}: {e}")
    else:
        print(f"Advertencia: El archivo '{file_path}' no fue encontrado.")

if not all_dfs:
    print("Error: No se cargó ningún archivo CSV de train o validation. Verifica las rutas y los archivos.")
    exit()

df_full = pd.concat(all_dfs, ignore_index=True)
print(f"Total de filas combinadas (train + validation): {len(df_full)}")
print("Primeras filas del DataFrame combinado:")

Configurando el modelo de embeddings...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
Modelo de embeddings configurado.

Cargando y combinando archivos CSV (train y validation)...
Archivo 'unfair_tos_data/unfair_tos_train.csv' cargado (filas: 5532).
Archivo 'unfair_tos_data/unfair_tos_validation.csv' cargado (filas: 2275).
Total de filas combinadas (train + validation): 7807
Primeras filas del DataFrame combinado:


In [12]:
df_full.head()

,text,labels
0,notice to california subscribers : you may can...,[]
1,"if you subscribed using your apple id , refund...",[]
2,"if you wish to request a refund , please visit...",[]
3,if you subscribed using your google play store...,[]
4,key changes in this version : we 've included ...,[]


In [13]:
# --- 2. Procesar Etiquetas y Crear Documentos LlamaIndex (METADATOS MODIFICADOS) ---

# Función para parsear las etiquetas
def parse_space_separated_label_list(label_str):
    if pd.isna(label_str) or not isinstance(label_str, str):
        return []
    cleaned_str = label_str.strip().strip('[]').strip()
    if not cleaned_str:
        return []
    try:
        return [int(num_str) for num_str in cleaned_str.split()]
    except ValueError:
        # Silenciamos la advertencia por cada parseo fallido para no llenar la consola si hay muchos
        # print(f"Advertencia: No se pudo parsear '{label_str}'. Se devuelve lista vacía.")
        return [] # Importante devolver lista vacía en caso de error

df_full['labels_list'] = df_full['labels'].apply(parse_space_separated_label_list)

In [14]:
df_full

,text,labels,labels_list
0,notice to california subscribers : you may can...,[],[]
1,"if you subscribed using your apple id , refund...",[],[]
2,"if you wish to request a refund , please visit...",[],[]
3,if you subscribed using your google play store...,[],[]
4,key changes in this version : we 've included ...,[],[]
...,...,...,...
7802,all claims filed or brought contrary to the di...,[],[]
7803,should either party file a claim contrary to t...,[],[]
7804,the following related policies are incorporate...,[],[]
7805,in the event of a conflict between this agreem...,[],[]


In [15]:
# Nombres de las etiquetas en el ORDEN CORRECTO
label_names_map = {
    0: 'Limitation of liability',
    1: 'Unilateral termination',
    2: 'Unilateral change',
    3: 'Content removal',
    4: 'Contract by using',
    5: 'Choice of law',
    6: 'Jurisdiction',
    7: 'Arbitration'
}

# Crear objetos Document de LlamaIndex
print("\nCreando objetos Document de LlamaIndex con metadatos modificados...")
llama_documents = []
for index, row in df_full.iterrows():
    clause_text = str(row['text'])
    label_indices = row['labels_list']
    human_readable_labels = [label_names_map[idx] for idx in label_indices if idx in label_names_map]

    # Metadatos Modificados: solo clause_id, labels_indices, labels_names
    metadata = {
        "clause_id": str(row.get('id', f"row_{index}")), # Usar 'id' si existe, si no un id generado
        "labels_indices": str(label_indices),
        "labels_names": ", ".join(human_readable_labels) if human_readable_labels else "Ninguna", # Si no hay etiquetas, poner "Ninguna"
    }
    doc = Document(text=clause_text, metadata=metadata)
    llama_documents.append(doc)

print(f"Se crearon {len(llama_documents)} Documentos LlamaIndex.")
if llama_documents:
    print("Ejemplo del primer Documento LlamaIndex (con metadatos modificados):")
    print(f"Texto: {llama_documents[0].text[:100]}...")
    print(f"Metadatos: {llama_documents[0].metadata}")


Creando objetos Document de LlamaIndex con metadatos modificados...
Se crearon 7807 Documentos LlamaIndex.
Ejemplo del primer Documento LlamaIndex (con metadatos modificados):
Texto: notice to california subscribers : you may cancel your subscription , without penalty or obligation ...
Metadatos: {'clause_id': 'row_0', 'labels_indices': '[]', 'labels_names': 'Ninguna'}


In [16]:
# --- 3. Crear el Índice Vectorial (Esto genera y almacena los embeddings) ---
print("\nCreando el índice vectorial (esto generará los embeddings)...")
if llama_documents:
    try:
        index = VectorStoreIndex.from_documents(llama_documents)
        print("Índice vectorial creado exitosamente.")

        # --- 4. Persistir el Índice para Uso Futuro ---
        # Cambiamos el nombre del directorio para reflejar que solo usa train y validation
        persist_directory = "./unfair_tos_kb_index_train_val"
        if not os.path.exists(persist_directory):
            os.makedirs(persist_directory)
        index.storage_context.persist(persist_dir=persist_directory)
        print(f"Índice persistido en: {persist_directory}")
        # ... (mensajes sobre cómo cargarlo) ...

    except Exception as e:
        print(f"Error creando o persistiendo el índice: {e}")
else:
    print("No hay documentos para crear el índice.")

print("\n--- Proceso de Creación de Knowledge Base (Train+Val) y Embeddings Completado ---")


Creando el índice vectorial (esto generará los embeddings)...
Índice vectorial creado exitosamente.
Índice persistido en: ./unfair_tos_kb_index_train_val

--- Proceso de Creación de Knowledge Base (Train+Val) y Embeddings Completado ---
